<a href="https://colab.research.google.com/github/rouven-spiess/AI_Makerspace_LLM_Engineering/blob/main/Copy_of_Generative_Supervised_Fine_tuning_of_GPT_2_(Assignment_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Supervised Fine-tuning of GPT-2

Now that we have our GPT-2 model all trained up - we need a way we can get it to generate what we want.

In the following notebook, we're going to use an approach called "Supervised Fine-tuning" to achieve our goals today.

In essence, we're going to use each example as a self-contained unit (with potential for something called "packing") and this is going to allow us to build "labeled" data.

For this notebook, we're going to be flying quite high up in the levels of abstraction. Take extra care to look into the libraries we're using today!

Let's start by grabbing our dependencies, as always:



In [1]:
!pip install transformers accelerate datasets trl bitsandbytes -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 12.8 MB/s eta 0:00:00


## Dataset Curation

We're going to be fine-tuning our model on SQL generation today.

First thing we'll need is a dataset to train on!

We'll use [this](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset today!

First up, let's load it and take a look at what we've got.

In [2]:
from datasets import load_dataset

sql_dataset = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
sql_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 78577
    })
})

In [4]:
sql_dataset['train'][0]

{'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?'}

So, we've got ~78.5K rows of:

- question - a natural language query about
- context - the `CREATE TABLE` statement - which gives us important context about the table
- answer - a SQL query that is aligned with both the question and the context.

Let's split our data into `train`, `val`, and `test` datasets.

We can use our `train` and `val` sets to train and evaluate our model during training - and our `test` set to ultimately benchmark the generations of our model!

In [5]:
sql_dataset_train_test = sql_dataset["train"].train_test_split(test_size=0.2)

In [6]:
sql_dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 62861
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 15716
    })
})

In [7]:
sql_dataset_val_test = sql_dataset_train_test["test"].train_test_split(test_size=0.5)

In [8]:
sql_dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
})

In [9]:
from datasets import DatasetDict

split_sql_dataset = DatasetDict({
    "train" : sql_dataset_train_test["train"],
    "val" : sql_dataset_val_test["train"],
    "test" : sql_dataset_val_test["test"]
})

In [10]:
split_sql_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 7858
    })
})

### Creating a "Prompt"

Now we need to create a prompt that's going to allow us to interact with our model when we desired the trained behaviour.

Think of this as a pattern that aligns the model with our desired outputs.

We need a single text prompt, as that is what the `SFTTrainer` we're going to use to fine-tune our model expects.

The basic idea is that we're going to merge the `question`, `context`, and `answer` into a single block of text that shows the model our desired outputs.

Let's look at what that block needs to look like:

```
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
```

Let's look at that from a completed prompt perspective to get a bit more information:

```
<|startoftext|>### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question.

### Input:
How many locations did the team play at on week 7?

### Context:
CREATE TABLE table_24123547_2 (location VARCHAR, week VARCHAR)

### Response:SELECT COUNT(location) FROM table_24123547_2 WHERE week = 7<|endoftext|>
```

As you can see, our prompt contains completed examples of our task. We're going to show our model many of these examples over and over again to teach it to produce outputs that are aligned with our goals!

First step, let's create a template we can use to call `.format()` on while constructing our prompts.

###🏗️Activity:

Create the following templates:

- `TEXT2SQL_TRAINING_PROMPT_TEMPLATE`
- `TEXT2SQL_INFERENCE_PROMPT_TEMPLATE`

> HINT: Remember that during inference we do not want to prepopulate the response.

In [11]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
"""

TEXT2SQL_INFERENCE_PROMPT_TEMPLATE =  """
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
"""

Now let's create a function we can map over our dataset to create the full prompt text block.

###🏗️Activity:

Define a `SYSTEM_MESSAGE` to use with your training data.

In [12]:
SYSTEM_MESSAGE = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question."""

In [13]:
def create_sql_prompt(sample):
  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"],
      response = sample["answer"]
  )

  return {"text" : full_prompt}

I've created this helper-function to be able to see how our model is doing visibly, rather than only through metrics.

In [14]:
def create_sql_prompt_and_response(sample):
  full_prompt = TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"]
  )

  ground_truth = sample["answer"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

Let's look at an example of a formatted prompt.

In [15]:
create_sql_prompt(split_sql_dataset["train"][0])

{'text': '\n<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\nYou must output the SQL query that answers the question.\n\n### Input:\nWhat is the word form for the ALA-LC transliteration of muay s″ain?\n\n### Context:\nCREATE TABLE table_name_55 (word_form VARCHAR, ala_lc VARCHAR)\n\n### Response:\nSELECT word_form FROM table_name_55 WHERE ala_lc = "muay s″ain"<|endoftext|>\n'}

Great!

Now we can map this over our dataset!

In [16]:
split_sql_dataset = split_sql_dataset.map(create_sql_prompt)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

## Load the Model And Preproccessing

Now for the moment we've all been waiting for...

Loading our model!

Let's use the `AutoModelForCausalLM` and `AutoTokenzier` classes from `transformers` to see just how easy this is.

- [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/auto#transformers.AutoModelForCausalLM)
- [`AutoTokenizer`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer)
- [GPT-2 Model Card](https://huggingface.co/gpt2)

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "gpt2"

gpt2_base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

gpt2_tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We need to make sure our tokenizer has a `pad_token` in order to be able to pad sequences so they're all the same length.

We'll use a little trick here to set our padding token to our eos (end of sequence) token to make training go a little smoother.

In [18]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

We also need to make sure we resize our model to be aligned with the token embeddings. If we didn't do this - we'd face a shape error while training!

In [19]:
gpt2_base_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

Now let's use the Hugging Face `pipeline` to see what generation looks like for our untrained model.

In [20]:
from transformers import pipeline, set_seed, GenerationConfig
generator = pipeline('text-generation', model=gpt2_base_model, tokenizer=gpt2_tokenizer)
set_seed(42)

def generate_sample(sample):
  prompt_package = create_sql_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=50,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      eos_token_id=gpt2_base_model.config.eos_token_id,
  )

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))
  print("+++++++++++++++")
  print("Ground Truth")
  print(prompt_package["ground_truth"])

In [21]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:

SELECT * FROM manufacturers WHERE name = 'S';

### Output:

SELECT * FROM manufacturers WHERE name = 'S';

### Context:

SELECT * FROM manufacturers WHERE name = 'S';

### Output:
+++++++++++++++
Ground Truth
SELECT founder FROM manufacturers WHERE name LIKE 'S%'


## Training the Model

Now that we have our model set up, our tokenizer set up, we can finally begin training!

We'll be using the `TrainingArguments` from Hugging Face's `transformers` library to help us keep track of our hyper-parameters. More information and documentation available:

- [`TrainingArguments`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

Let's look at our Trainer, and set some hyper-parameters:

- `per_device_train_batch_size` - this is a batch size that accomodates distributed training
- `gradient_accumulation_steps` - this is exactly the same as the previous notebook, it's a way to "simulate" a large batch size by collecting losses over multiple iterations - scaling them - and then combining them together.
- `gradient_checkpointing` - I'll let the authors speak for themselves [here](https://github.com/cybertronai/gradient-checkpointing). In essence: This saves memory at the cost of computational time.
- `max_grad_norm` - this is the value used for gradient clipping, which is a method of reducing vanishing gradient potential
- `max_steps` - how many steps will we train for?
- `learning_rate` - how fast should we learn?
- `save_total_limit` - how many versions of the model will we save?
- `logging_steps` - how often we should log
- `output_dir` - where to save our checkpoints
- `optim` - which optimizer to use, you'll notice we're using a full precision paged optimizer - this is a performative and stable optimizer - but it uses extra memory
- `lr_scheduler_type` - we are once again using a cosine scheduler!
- `evaluation_strategy` - we have an evaluation dataset, this defines when we should leverage it during training
- `eval_steps` - how many steps we should evaluate for
- `warmup_ration` - how many "warmup" steps we take to reach our full learning rate before we start decaying. this is a ration of our max_steps

In [22]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing =True,
 max_grad_norm= 0.3,
 ###num_train_epochs=2,
 max_steps=500,
 learning_rate=2e-4,
 save_total_limit=3,
 logging_steps=10,
 output_dir="sql_gpt2",
 optim="paged_adamw_32bit",
 lr_scheduler_type="cosine",
 evaluation_strategy="steps",
 eval_steps=50,
 warmup_ratio=0.05,
)

###❓Question

Is this process using usupervised, or supervised learning?

RESPONSE: Supervised learning as we provide the labels during training.

Now, for our `SFTTrainer` AKA "Where the magic happens".

You can read all about the `SFTTrainer` here:

- [`SFTTrainer`](https://huggingface.co/docs/trl/sft_trainer#trl.SFTTrainer)

This `SFTTrainer` is going to take our above training arguments, our data, our model and our tokenizer, and train it all for us!

Notice that we're setting `max_seq_length` to the maximum context window of our model - this ensures we do not exceed our maximum context window, and will pad our examples up to the maximum context window!

In [23]:
trainer = SFTTrainer(
 gpt2_base_model,
 dataset_text_field="text",
 train_dataset=split_sql_dataset["train"],
 eval_dataset=split_sql_dataset["val"],
 tokenizer=gpt2_tokenizer,
 max_seq_length=1024,
 args=training_args
)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

###❓Question

What do we use to determine loss in fine-tuning GPT-2?

> HINT: [This](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py) can help you if you get stuck!

Finally, we can call our `.train()` method and watch it go!

In [24]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.731300,0.678428
100,0.642300,0.626335
150,0.641200,0.596237
200,0.603000,0.577616
250,0.570400,0.561969
300,0.590200,0.551018
350,0.557800,0.541943
400,0.545200,0.537120
450,0.557100,0.534850
500,0.543700,0.534208


TrainOutput(global_step=500, training_loss=0.6631961879730225, metrics={'train_runtime': 445.9515, 'train_samples_per_second': 17.939, 'train_steps_per_second': 1.121, 'total_flos': 655351036416000.0, 'train_loss': 0.6631961879730225, 'epoch': 0.13})

Let's save our fine-tuned model!

In [25]:
trainer.save_model()

## Testing our Model

Now that we have a fine-tuned model, let's see how it did

In [26]:
ft_gpt2_model = AutoModelForCausalLM.from_pretrained("sql_gpt2")

In [27]:
generator = pipeline('text-generation', model=ft_gpt2_model, tokenizer=gpt2_tokenizer, )

In [28]:
generate_sample(split_sql_dataset["test"][1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:
SELECT MAX(fatalities) FROM table_name_62 WHERE aircraft_type = "lockheed l-1049g" AND aircraft_type = "l-1049g" AND fatalities = "1" AND aircraft_type =
+++++++++++++++
Ground Truth
SELECT SUM(fatalities) FROM table_name_62 WHERE aircraft_type = "lockheed l-1049g"


That is *significantly* better.

#### ###❓Question

How might you evaluate your generated SQL? Please provide 3 different methods.

RESPONSE:
- Check if both, models response and ground truth return the same result with a constructed data table
- Translate both expressions to first order logic and then check if they are equivalent using a proving system
- Using another LLM trained for SQL expression equivalence

All of the above could then be evaluated with a binary crossentropy loss.